In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
from os import system
import json
import os

In [2]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device, torch.cuda.get_device_name(device))

SOS_token = 0
EOS_token = 1

hidden_size = 256
vocab_size = 28
teacher_forcing_ratio = 0.8
LR = 0.05

cuda:2 NVIDIA GeForce RTX 2080 Ti


In [6]:
class Dataloader():
    def __init__(self, path, useonehot=False):
        self.data_path = path
        self.useonehot = useonehot
        self.data = []
        self.vocab_table_idx2word = {0: "SOS", 1: "EOS"}
        self.vocab_table_word2idx = {}

        # read json
        with open(self.data_path) as f:
            data_json = json.load(f)
        # combine inputs and labels
        for ele in data_json:
            inputs = ele["input"]
            label = ele["target"]
            for ins in inputs:
                self.data.append([ins, label])

        # make vocab table
        for idx, ele in enumerate("abcdefghijklmnopqrstuvwxyz"):
            self.vocab_table_idx2word[idx+2] = ele
        self.vocab_table_word2idx = {v: k for k, v in self.vocab_table_idx2word.items()}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        idx = idx % len(self.data)

        input = self.data[idx][0]
        label = self.data[idx][1]

        # char to idx
        idx_input = np.array([0] + [self.vocab_table_word2idx[ele] for ele in input] + [1])
        idx_label = np.array([0] + [self.vocab_table_word2idx[ele] for ele in label] + [1])
        # idx to one-hot
        if self.useonehot:
            onehot_input = np.zeros((idx_input.size, vocab_size), dtype=np.int32)
            onehot_input[np.arange(idx_input.size), idx_input] = 1
            onehot_label = np.zeros((idx_label.size, vocab_size), dtype=np.int32)
            onehot_label[np.arange(idx_label.size), idx_label] = 1

            return torch.from_numpy(onehot_input).unsqueeze(1).to(device), torch.from_numpy(onehot_label).unsqueeze(1).to(device)
        else:
            return torch.from_numpy(idx_input.reshape(-1, 1)).to(device), torch.from_numpy(idx_label.reshape(-1, 1)).to(device)


In [4]:
dl = Dataloader("./train.json")
print(dl[0][0].shape)

torch.Size([9, 1])


In [7]:
dl = Dataloader("./train.json", useonehot=True)
print(dl[0][0].shape)

torch.Size([9, 1, 28])


In [1]:
import nltk

ModuleNotFoundError: No module named 'nltk'